# Violando as hipóteses do MLC no Python

- https://www.youtube.com/watch?v=Nh1RMs2pL0o

Vamos aqui tomar emprestado o arquivo ['card.xlsx'](https://docs.google.com/spreadsheets/d/1rj0LBjPoAQ8lI8nV72deOvUxzZs5s_Ob/edit#gid=107230395), que contém o banco de dados do artigo de David Card (1995) *Using Geographic Variation in College Proximity to Estimate the Return to Schooling*. 

O arquivo contém um subgrupo dos dados originais, com 29 variáveis, $n=2214$ observações. 

Queremos estimar o seguinte modelo:
$$\text{log}(Wage)_i = \beta_0 + \beta_1 Educ_i + \beta_2 Exper_i + \beta_3 Exper_i^2 + \beta_4 South_i + \beta_5 Black_i + \epsilon_i,$$

Assim, começamos montando a matriz X e o vetor y de dados.

Note que precisamos temos funções das variáveis $Wage$ e $Exper$ para calcular.

Além disso, precisamos incluir a constante no vetor X.

In [1]:
from matplotlib.pyplot import figure, plot, bar, draw, hist, legend, draw , subplots
#from numpy.random import randn, rand, standard_t, normal, uniform
from numpy import sqrt, arange, mean, std
from numpy.linalg import inv
import pandas as pd                                             ## Carrega o pacote pandas e o chama de pd
import numpy as np                                              ## Carrega o pacote pandas e o chama de pd
from IPython.display import Math #FK para escrever equações 


link = 'https://doc-14-20-sheets.googleusercontent.com/export/v8f47es1ueddr6v906ehiub340/jc91mptfbiosdklpmsqs90v1lo/1650744105000/118035206058290454508/108913105810215373705/1rj0LBjPoAQ8lI8nV72deOvUxzZs5s_Ob?format=xlsx&id=1rj0LBjPoAQ8lI8nV72deOvUxzZs5s_Ob&dat=AFCstmqwLJ7IO_d6-DXWsrc7OQouLJrPF_bHvIpRnRz_1iA1L1TqgVpzQPF9Elz7OLsOEqNgepLe96IyRVJtj3gqrbBRqOt-3Wm_bBcT3hrbbOLHQNC7fPyMnQHpPERFtGeAgQzGrKlaKEiyxezXz8gYWOCJNBEE6BcS5pKtOtuyp6BAVbTegk-4TMlda8bOrKHAWfWd8vVWhLaT_0jhVlMx2WvHGyxxkr1JA4A0kxnvBHVvdVE4Uu6E6bUCwUC-AHw8KJuLAPN8jVyKxN_Di3SbsghpujfHBjyWG0TWHhvAMH2kdwMA06La_u4eaPTfTXd5Jz3sw_ArDNV8tiTpOcmZkTPDRW5AJh5KngkN3Jly72OVTTUL-YiRpN_lARDrmNI05ZRQIG9eKbqqydyMay6ioiaErZDECSZfLb-Bjb8oyUklpjc53mOXZj4U_gDAJCv7BmDlGE-cXtRK5WidHYXknPVIc9YzDGj2iyJj4kdd8TT-EqV9DjQkmsX0KV-gpNBjZgCp8EnIz4FgcnV2qArDkqhIse9MXEvrdCKNj-bii3bxlOSUk7Msr2iAjp2ygZlNq_F7x-dz2x8NjVkA6y9QTYUVCe7oheSR16SI09W2dIFyldgwItSfvPE-80O8uswf1j7fte9gCMR0rcY4Qt6PvFzVbxJ9C2-RzcBn5-BdsZ7KjcbK65wandiHMJgfa3qrFh6LxU0s0qVXXRwCmtqyJTQhxsi8AwBsp6Yip6CKXgwqA__1iO4'
dados = pd.read_excel('card.xlsx')
print(dados.shape)
dados.tail()

(2215, 29)


,id,near2,near4,Educ,Unnamed: 4,fatheduc,motheduc,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Black,Unnamed: 22,South,Unnamed: 24,Wage,Unnamed: 26,Unnamed: 27,Exper
2210,5200,0,1,13,30,4,9,117386,1,0,...,0,1,1,1,0,0,529,0,1,11
2211,5206,0,1,13,27,6,8,91831,1,0,...,0,1,0,0,1,0,500,0,1,8
2212,5210,0,1,15,29,16,18,119965,1,0,...,0,1,0,1,1,0,712,0,1,8
2213,5211,0,1,16,27,16,18,91831,1,0,...,0,1,0,1,1,0,472,0,1,5
2214,5218,0,1,12,25,8,12,82135,1,0,...,0,1,0,0,1,0,335,0,1,7


In [2]:
# by kfouri, para ser usado em testes Variaveis instrumentais
dados['wage_log'] = np.log(dados["Wage"])
dados['Exper2'] = dados["Exper"]**2  
dados['beta_zero'] = 1
dados.tail()

,id,near2,near4,Educ,Unnamed: 4,fatheduc,motheduc,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,South,Unnamed: 24,Wage,Unnamed: 26,Unnamed: 27,Exper,wage_log,Exper2,beta_zero
2210,5200,0,1,13,30,4,9,117386,1,0,...,1,0,0,529,0,1,11,6.270988,121,1
2211,5206,0,1,13,27,6,8,91831,1,0,...,0,1,0,500,0,1,8,6.214608,64,1
2212,5210,0,1,15,29,16,18,119965,1,0,...,1,1,0,712,0,1,8,6.568078,64,1
2213,5211,0,1,16,27,16,18,91831,1,0,...,1,1,0,472,0,1,5,6.156979,25,1
2214,5218,0,1,12,25,8,12,82135,1,0,...,0,1,0,335,0,1,7,5.814131,49,1


## adaptando a variaveis

lembrando: $$\text{log}(Wage)_i = \beta_0 + \beta_1 Educ_i + \beta_2 Exper_i + \beta_3 Exper_i^2 + \beta_4 South_i + \beta_5 Black_i + \epsilon_i,$$

In [3]:
listanomes = ["Educ","Exper","South","Black"]                   # Cria lista de variáveis
Dados      = np.array(dados[listanomes])                        # Seleciona os dados e salva numa matriz 
y          = np.array(dados["Wage"])                            # Seleciona a variável dependente
dados[listanomes].head()

,Educ,Exper,South,Black
0,12,9,0,0
1,12,16,0,0
2,11,10,0,0
3,12,16,0,0
4,12,8,0,0


Adaptando o y para ser $\log(\text{Wage})_i$

In [4]:
print(y)
y = np.log(y)                                                   # Converte o nível do salário em log(salário) (LN)
print(y)
n = len(y)                                                      # Obtém o número de observações
y = np.reshape(y,(n,1)) 
print(y)

[481 721 250 ... 712 472 335]
[6.17586727 6.58063914 5.52146092 ... 6.56807791 6.15697899 5.81413053]
[[6.17586727]
 [6.58063914]
 [5.52146092]
 ...
 [6.56807791]
 [6.15697899]
 [5.81413053]]


Adaptando o termo $\beta_3 Exper_i^2$

In [5]:
print(dados["Exper"].to_numpy())
Exper2 = np.array(dados["Exper"])**2                              # Seleciona Exper e eleva ao quadrado
print(Exper2)
Exper2 = Exper2[...,None]                                         # TEM A MESMA FUNCAO QUE O RESHAPE
print(Exper2)                  

[ 9 16 10 ...  8  5  7]
[ 81 256 100 ...  64  25  49]
[[ 81]
 [256]
 [100]
 ...
 [ 64]
 [ 25]
 [ 49]]


Montando a variavel X

In [6]:
def get_X():   
    X_tmp1 = Dados[:,0:2]                                             # Seleciona as variáveis Educ e Exper  
    constante_matriz_1 = np.ones((n,1))
    
    X_tmp1 = np.hstack( [constante_matriz_1, X_tmp1] )                # Concatena a constante na primeira coluna (Beta zero)
    X_tmp1 = np.concatenate( (X_tmp1, Exper2 ), axis=1)               # Concatena X_tmp1 para agregar Exper^2
    return np.concatenate( (X_tmp1,Dados[:,2:]), axis=1)              # Concatena X_tmp1 com "South","Black"

X = get_X()
X

array([[  1.,  12.,   9.,  81.,   0.,   0.],
       [  1.,  12.,  16., 256.,   0.,   0.],
       [  1.,  11.,  10., 100.,   0.,   0.],
       ...,
       [  1.,  15.,   8.,  64.,   1.,   0.],
       [  1.,  16.,   5.,  25.,   1.,   0.],
       [  1.,  12.,   7.,  49.,   1.,   0.]])

Assim, atualizamos a lista de variávels para:

In [7]:
listanomes = ["Const","Educ","Exper","Exper^2","South","Black"]     # Atualiza a lista de variáveis
k          = len(X[0])                                              # Obtém o número de variáveis/colunas

Vamos começar pela estimação OLS de $\beta = (\beta_0,\beta_1,\beta_2,\beta_3, \beta_4, \beta_5)'$, que denotaremos por $\beta_{ols}$.

# Estimação OLS de $\beta$

## Modelo na Raça

In [8]:
Beta_ols= inv(X.T @ X) @ (X.T @ y)                  # Estimativa OLS para Beta
e_hat   = y - X @ Beta_ols;                         # resíduo da regressão i
s2_ols  = (e_hat.T @ e_hat)/(n-k);                  # estimativa de s2 para a amostra i
AVAR_ols= s2_ols * inv(X.T @ X);                    # Estimando a Variância esférica
dp      = sqrt(np.diag(AVAR_ols));                  # Computa o desvio padrão
tsta_ols= Beta_ols.T/dp                             # Computa a estatística t.

In [9]:
D       = np.diag(e_hat[:,0])**2                    # Para utilizar o comando diag, precisamos converter e_hat em nx1. 
Whit_ols= inv(X.T @ X) @ X.T @ D @ X @ inv(X.T @ X) # Matriz Robusta de White    
se_r_ols= sqrt(np.diag(Whit_ols));                  # SE_robustos
t_r_ols = Beta_ols.T/se_r_ols;                      # Estatística t com SE_Robusto

R2=1-((e_hat.T@e_hat)/n)/np.var(y)                  # R_2

R2_adj=1-((e_hat.T@e_hat)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj;                                             #R_2 Ajustado

print('-----------------------------------------------------------------------------');
print('                                Estimação OLS');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '    Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_ols[0],    '    Variável dependente  ', "log(Wage)")
print('---------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat     S.E. Rob     t-Rob')
for i in range(k):
    tuplaprint=(Beta_ols[i],dp[i],tsta_ols[0,i],se_r_ols[i],t_r_ols[0,i])                     # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %8.4f    %8.4f    %8.4f" % tuplaprint)  # imprime cada coeficiente

print('\n---------------------------------------------------------------------------')


-----------------------------------------------------------------------------
                                Estimação OLS
-----------------------------------------------------------------------------
R2:     [0.23730627]     Graus de liberdade    2209
R2_adj: [0.23557993]     Número de Observações 2215
s^2:    [0.14760106]     Variável dependente   log(Wage)
---------------------------------------------------------------------------
               Coef        S.E.       t-Stat     S.E. Rob     t-Rob
     Const    4.7351      0.0796     59.4716      0.0837     56.5519
      Educ    0.0802      0.0041     19.4582      0.0043     18.5463
     Exper    0.0893      0.0081     11.0743      0.0081     10.9738
   Exper^2   -0.0025      0.0004     -6.0645      0.0004     -6.0905
     South   -0.1359      0.0178     -7.6560      0.0177     -7.6730
     Black   -0.1591      0.0238     -6.6918      0.0238     -6.6928

---------------------------------------------------------------------------


## Modelo Statsmodels
Vamos comparar com os resultados do pacote do statsmodels

In [10]:
import statsmodels.api as sm                                            ## Carrega o statsmodels como sm
# from statsmodels.sandbox.regression.predstd import wls_prediction_std   ## 
ols_model = sm.OLS(y,X)
ols_results = ols_model.fit()
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     137.5
Date:                Tue, 17 May 2022   Prob (F-statistic):          3.65e-127
Time:                        23:39:48   Log-Likelihood:                -1021.0
No. Observations:                2215   AIC:                             2054.
Df Residuals:                    2209   BIC:                             2088.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7351      0.080     59.472      0.0

## OLS FK

In [11]:
def temp():
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error    
    import statsmodels.api as sm
    
    # Preciso do treino e teste aqui para capturar r2
    lr_modelo = LinearRegression()
    lr_modelo.fit(X=X, y=y)
    
#     print('R² = %s' % metrics.r2_score(y_test, y_previsto).round(2))
    
    ols_model = sm.OLS(y,X)
    ols_results = ols_model.fit()
    print(ols_results.summary())


# Estimação FGLS de $\beta$

- **GLS**: Generalized least squares - *Mínimos quadrados generalizados (MQG)*
- **FGLS**: Feasible Generalized least squares - *Mínimos quadrados generalizados factivel (MQGF)*
- **WGLS**: Weighted least squares - *Mínimos quadrados ponderados*

Vamos agora estimar $\beta$ utilizando o estimador de dois estágios FGLS. Procederemos o seguinte algorítmo:

1. Estime a equação $y_i=x_i'\beta + \epsilon_i$ por OLS e compute os resíduos, $e_i$.
1. Regrida $e_i^2$ sobre $x_i$ e obtenha uma estimativa OLS para $\hat{\alpha}$, que está associado à regressão $$e_i^2 =x_i'\alpha + u_i$$

1. Divida $y_i$ e $x_i$ por $\tfrac{1}{\sqrt{x_i'\hat{\alpha}}}$ e estime o modelo por WLS usando a equação $\tilde{y}_i=\tilde{x}_i\beta +\tilde{\epsilon}_i$.

1.Alternativamente, compute $$\widehat{\beta}(V) = (X'(V(\hat{\alpha}))^{-1}X)^{-1}X'(V(\hat{\alpha}))^{-1}y,$$
    em que
    $$V_{(n \times n)}(\hat{\alpha})=\begin{bmatrix}
x_1'\hat{\alpha} & \cdots  &0 \\ 
\vdots & \ddots & \vdots\\ 
0 & \cdots & x'_n\hat{\alpha}
\end{bmatrix}$$

Como já temos $\beta_{ols}$ e $e_i$ da regressão acima, basta prosseguimos com os passos 2 e 3 ou 2 e 4. Lembre-se que, após a transformação, $s^2=1$ dada a normalização.

Além disso, 
$$\widehat{\text{Avar}(\widehat{\beta}(V)|X)}=\Bigg(X'V^{-1}X\Bigg)^{-1}$$

In [12]:
## Obtendo alpha- relativo a regrecao do passo 2 :
alfafit  = X @ inv(X.T @ X) @ (X.T @ (e_hat**2))         # Obtém o valor predito usando a matriz de projeção
alfafit

array([[0.1466638 ],
       [0.13791917],
       [0.14528563],
       ...,
       [0.14742999],
       [0.14918608],
       [0.14631326]])

Note que a matriz V é, na verdade, $\text{diag}(X\hat{\alpha})$.

Diagonalizar o alfafit

In [13]:
V       = np.matrix(alfafit)      # Para isso, utilizamos o comando matrix, que converte os dados em matriz de formatos arbitrários
V       = np.diag(V.A1)           # e extraímos a "matriz 1-D" do vetor e_hat. Em seguida, elevamos ao quadrado cada elemento
V

array([[0.1466638 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.13791917, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.14528563, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.14742999, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.14918608,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.14631326]])

In [14]:
y_tilde   = y/(alfafit**.5) 
X_tilde   = X/(alfafit**.5) 

- $\widehat{\beta}(V) = (X'(V(\hat{\alpha}))^{-1}X)^{-1}X'(V(\hat{\alpha}))^{-1}y$ 
- $\tilde{y}_i=\tilde{x}_i\beta +\tilde{\epsilon}_i$

In [15]:
Beta_gls  = inv(X.T @ inv(V) @ X) @ (X.T @ inv(V) @ y)         # Estimação FGLS
Beta_gl1  = inv(X_tilde.T @ X_tilde)  @ (X_tilde.T @ y_tilde)  # Estimação FGLS - feito 2x para confirmar
e_tilde   = y_tilde - X_tilde @ Beta_gls;                      # resíduo da regressão i
s2_gls    = (e_tilde.T @ e_tilde)/(n-k);                       # estimativa de s2 para a amostra i (desvio padrao)
AVAR_gls  = inv(X.T @ inv(V) @ X);                             # Estimando a Variância esférica
dp        = sqrt(np.diag(AVAR_gls));                           # Computa o desvio padrão
tsta_gls  = Beta_gls.T/dp                                      # Computa a estatística t.
#D        = np.diag(e_tilde[:,0])**2                           # Eleva o resíduo ao quadrado e diagonaliza o vetor resultante
AVA2_gls  = inv(X_tilde.T @ X_tilde)                           # Mesma de antes. Apenas confirmando o resultado. Não é White!
se_r_gls  = sqrt(np.diag(AVA2_gls));                          
t_r_gls   = Beta_gls.T/se_r_gls;

R2=1-((e_tilde.T@e_tilde)/n)/np.var(y)
R2
R2_adj=1-((e_tilde.T@e_tilde)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj

print('-----------------------------------------------------------------------------');
print('                                Estimação WGLS');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '    Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_gls[0],    '     Variável dependente  ', "log(Wage)")
print('-----------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat       S.E. Rob     t-Rob')
for i in range(k):
    tuplaprint=(Beta_gls[i],dp[i],tsta_gls[0,i],se_r_gls[i],t_r_gls[0,i])                     # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %9.4f    %9.4f    %9.4f" % tuplaprint)  # imprime cada coeficiente

print('------------------------------------------------------------------------------')

-----------------------------------------------------------------------------
                                Estimação WGLS
-----------------------------------------------------------------------------
R2:     [-4.18114865]     Graus de liberdade    2209
R2_adj: [-4.19287602]     Número de Observações 2215
s^2:    [1.00268691]      Variável dependente   log(Wage)
-----------------------------------------------------------------------------
               Coef        S.E.       t-Stat       S.E. Rob     t-Rob
     Const    4.7433      0.0793      59.8479       0.0793      59.8479
      Educ    0.0799      0.0041      19.4451       0.0041      19.4451
     Exper    0.0882      0.0080      11.0605       0.0080      11.0605
   Exper^2   -0.0024      0.0004      -6.0403       0.0004      -6.0403
     South   -0.1367      0.0177      -7.7244       0.0177      -7.7244
     Black   -0.1620      0.0239      -6.7850       0.0239      -6.7850
-----------------------------------------------------

In [16]:
print(e_hat)
print(e_tilde)

[[-0.12663342]
 [ 0.08258106]
 [-0.74351029]
 ...
 [ 0.20846977]
 [-0.11062708]
 [-0.2523946 ]]
[[-0.33113851]
 [ 0.21349058]
 [-1.95198091]
 ...
 [ 0.54629113]
 [-0.28471282]
 [-0.65894137]]


# Endogeneidade

- TSLS: Two Stage Least Squares

Vamos assumir agora que $Educ$ é endógena, mas que temos um bom instrumento pela variável $near4$. 
Assim, vamos estimar o modelo com uma variável endógena e um instrumento, configurando o caso exatamente identificado.

Ou seja, para o indivíduo $i$, o modelo é:

\begin{equation}
y_i=x_i'\beta +u_i
\end{equation}
em que $$\underbrace{x_i}_{(K\times 1)} = (1, \underbrace{Educ_i}_{\text{endógena}}, Exper_i, Exper_i^2, South_i, Black_i)$$ 

e o **vetor de instrumentos** $$\underbrace{z_i}_{(L=K\times 1)} = (1, \underbrace{near4_i}_{\text{instrumento}}, Exper_i, Exper_i^2, South_i, Black_i)$$


Assim, o primeiro passo é construir o vetor $Z$. Neste caso é simples, pois ele é muito parecido com X. Apenas precisamos trocar os dados na coluna do X que corresponde à variável Educ pelos dados de near4. Isso é feito a seguir:

In [17]:
import copy
Z      = copy.deepcopy(X);              # Este comando é necessário, pois Z=X cria uma nova referência à X e não uma cópia de X
Z[:,1] = np.array(dados["near4"])       # substitui a coluna 1 por near4

In [18]:
Z

array([[  1.,   0.,   9.,  81.,   0.,   0.],
       [  1.,   0.,  16., 256.,   0.,   0.],
       [  1.,   1.,  10., 100.,   0.,   0.],
       ...,
       [  1.,   1.,   8.,  64.,   1.,   0.],
       [  1.,   1.,   5.,  25.,   1.,   0.],
       [  1.,   1.,   7.,  49.,   1.,   0.]])

In [19]:
X

array([[  1.,  12.,   9.,  81.,   0.,   0.],
       [  1.,  12.,  16., 256.,   0.,   0.],
       [  1.,  11.,  10., 100.,   0.,   0.],
       ...,
       [  1.,  15.,   8.,  64.,   1.,   0.],
       [  1.,  16.,   5.,  25.,   1.,   0.],
       [  1.,  12.,   7.,  49.,   1.,   0.]])

## Estimador de variaveis instrumentais
Agora que sabemos que estamos ok, basta-nos adaptarmos os códigos que já temos, pois 
\begin{equation}
    \hat{\beta}_{IV} = \bigg( \frac{1}{n}\sum_{i=1}^n z_i x_i'\Bigg)^{-1} \Bigg( \frac{1}{n}\sum_{i=1}^n z_i y_i\Bigg) = (Z'X)^{-1}(Z'y)
\end{equation}
em que $Z$ e $X$ são $n \times K$ e $y$ é $n \times 1$.

<span style='color:red'>Aqui como so tem uma variavel instrumental, podemos utilizar o estimador mais simples. Não precisa utilizar uma matriz de ponderação.</span>


E, sob variância esféria,
<span style='color:red'>Quando há homoscedasticidade</span>

$$\widehat{\text{Avar}(\hat{\beta}_{IV})} =s^2 (Z'X)^{-1} (Z'Z) (X'Z)^{-1} $$

ou, c.c., <span style='color:red'>Quando HÁ heterocedasticidade, utiliza a matriz D (White) </span>

$$\widehat{\text{Avar}(\hat{\beta}_{IV})} = (Z'X)^{-1} (Z'\hat{D} Z) (X'Z)^{-1} $$


In [20]:
def matriz_D_metodo1(e):
    '''
    Matriz D que na diagnoal tem o residuo da regressao ao quadrado
    '''
    n = len(e)
    D       = np.zeros((n,n))                     # Matriz D que na diagnoal tem o residuo da regressao ao quadrado
    for i in range(n):
        D[i,i] = e_hat[i,0]**2
        
    return D

In [21]:
def matriz_D_metodo2(e):
    '''
    Matriz D que na diagnoal tem o residuo da regressao ao quadrado
    '''       
    return np.diag(e[:,0])**2 

In [22]:
Beta_iv = inv(Z.T @ X) @ (Z.T @ y)            # Estimador para IV com L=K 
e_hat   = y-X@Beta_iv;                        # resíduo da regressão i
s2_iv   = (e_hat.T@e_hat)/(n-k);              # estimativa de s2 para a amostra i (desvio padrao)
AVAR_iv = s2_iv*inv(Z.T@X)@Z.T@Z@inv(X.T@Z);  # Estimando a Variância esférica
dp      = sqrt(np.diag(AVAR_iv));             # Computa o desvio padrão
tsta_iv = Beta_iv.T/dp                        # Computa a estatística t.
D       = np.zeros((n,n))                     # Matriz D que na diagnoal tem o residuo da regressao ao quadrado
for i in range(n):
    D[i,i] = e_hat[i,0]**2
    
Whit_iv = inv(Z.T@X)@(Z.T@D@Z)@inv(X.T@Z)     # Variancia (White) usando a matriz D, leva em conta a heterocedasticidade
se_r_iv = sqrt(np.diag(Whit_iv));
t_r_iv  = Beta_iv.T/se_r_iv;

R2=1-((e_hat.T@e_hat)/n)/np.var(y)
R2
R2_adj=1-((e_hat.T@e_hat)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj

print('-----------------------------------------------------------------------------');
print('                                Estimação IV');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '    Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_iv[0],    '    Variável dependente  ', "log(Wage)")
print('-----------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat     S.E. Rob     t-Rob')
for i in range(k):
    tuplaprint=(Beta_iv[i],dp[i],tsta_iv[0,i],se_r_iv[i],t_r_iv[0,i])                     # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %8.4f    %8.4f    %8.4f" % tuplaprint)  # imprime cada coeficiente

print('-----------------------------------------------------------------------------')


-----------------------------------------------------------------------------
                                Estimação IV
-----------------------------------------------------------------------------
R2:     [-0.17483072]     Graus de liberdade    2209
R2_adj: [-0.17748991]     Número de Observações 2215
s^2:    [0.22736028]     Variável dependente   log(Wage)
-----------------------------------------------------------------------------
               Coef        S.E.       t-Stat     S.E. Rob     t-Rob
     Const    2.2659      1.0254      2.2097      0.9999      2.2661
      Educ    0.2226      0.0591      3.7676      0.0577      3.8610
     Exper    0.1507      0.0273      5.5255      0.0270      5.5853
   Exper^2   -0.0027      0.0005     -5.2476      0.0006     -4.7070
     South   -0.0856      0.0303     -2.8270      0.0294     -2.9101
     Black   -0.0359      0.0589     -0.6097      0.0562     -0.6391
----------------------------------------------------------------------------

- *O S.E. e t-Stat sao relativos a hipotese de homoscedaisticidade e os Rob (robustos) de heteroscedasticidade.*
- *O desvio padrao do robusto é menor que o nao robusto. Maior eficiencia ao acomodar a heteroscedasticidade.*
- *somente a variavel Black possui significancia estatistica (Regra de boulos???) em modulo maior que 1.96*



OBSERVACAO EXTERNA

Quando a amostra não possui muitas observações (por exemplo, < 30 observações), pode-se refinar a aproximação utilizando-se a distribuição t-Student, ao invés da distribuição normal.

A distribuição t-Student também é simétrica, com média 0, mas possui caudas mais pesadas do que a distribuição normal.

[ref](https://repositorio.enap.gov.br/bitstream/1/3452/4/Aula%203a%20-%20Geraldo%20Goes%20e%20Alexandre%20Ywata%20-%20Regressao_Linear_v2.pdf)

## Usando o statsmodels
Vamos então confirmar os resultados no Python.

Para isso, carregamos os pacotes de estimação GMM:

<span style='color:red'>y variavel explicada<br> X as variaveis explicativas<br> Z a matriz de instrumentos </span>

In [27]:
Z

array([[  1.,   0.,   9.,  81.,   0.,   0.],
       [  1.,   0.,  16., 256.,   0.,   0.],
       [  1.,   1.,  10., 100.,   0.,   0.],
       ...,
       [  1.,   1.,   8.,  64.,   1.,   0.],
       [  1.,   1.,   5.,  25.,   1.,   0.],
       [  1.,   1.,   7.,  49.,   1.,   0.]])

In [30]:
from statsmodels.sandbox.regression.gmm import IV2SLS
IV_model = IV2SLS(y,X,Z)
IV_results = IV_model.fit()
print(IV_results.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                      -0.175
Model:                         IV2SLS   Adj. R-squared:                 -0.177
Method:                     Two Stage   F-statistic:                     42.92
                        Least Squares   Prob (F-statistic):           2.49e-42
Date:                Wed, 18 May 2022                                         
Time:                        00:39:36                                         
No. Observations:                2215                                         
Df Residuals:                    2209                                         
Df Model:                           5                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2659      1.025      2.210      0.0

## Usando o statsmodels 2

In [31]:
from statsmodels.sandbox.regression.gmm import IV2SLS

endog = dados[["wage_log"]]
exog = dados[['beta_zero', 'Educ', 'Exper', 'Exper2', 'South', 'Black']]
instrument = dados[['beta_zero', 'near4', 'Exper', 'Exper2', 'South', 'Black']]
results = IV2SLS(endog=endog,exog=exog,instrument=instrument).fit()
print(results.summary())

                          IV2SLS Regression Results                           
Dep. Variable:               wage_log   R-squared:                      -0.175
Model:                         IV2SLS   Adj. R-squared:                 -0.177
Method:                     Two Stage   F-statistic:                     42.92
                        Least Squares   Prob (F-statistic):           2.49e-42
Date:                Wed, 18 May 2022                                         
Time:                        00:39:39                                         
No. Observations:                2215                                         
Df Residuals:                    2209                                         
Df Model:                           5                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
beta_zero      2.2659      1.025      2.210      0.0

## linearmodels.IV2SLS - Homoskedastic

https://bashtage.github.io/linearmodels/iv/index.html

In [28]:
import linearmodels
# from linearmodels import IV2SLS

dependent = dados[["wage_log"]]
endog = dados[["Educ"]]
exog = dados[['beta_zero', 'Exper', 'Exper2', 'South', 'Black']]
instrument = dados[['near4']]

IV_model = linearmodels.IV2SLS(dependent = dependent, endog=endog,exog=exog,instruments=instrument)
IV_results = IV_model.fit(cov_type="unadjusted")

print(IV_results)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               wage_log   R-squared:                     -0.1748
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1775
No. Observations:                2215   F-statistic:                    215.18
Date:                Tue, May 17 2022   P-value (F-stat)                0.0000
Time:                        23:46:39   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
beta_zero      2.2659     1.0240     2.2127     0.0269      0.2588      4.2729
Exper          0.1507     0.0272     5.5330     0.00

## linearmodels.IV2SLS - Heteroskedastic

In [29]:
import linearmodels

dependent = dados[["wage_log"]]
endog = dados[["Educ"]]
exog = dados[['beta_zero', 'Exper', 'Exper2', 'South', 'Black']]
instrument = dados[['near4']]

IV_model = linearmodels.IV2SLS(dependent = dependent, endog=endog,exog=exog,instruments=instrument)
IV_results = IV_model.fit(cov_type="robust")

print(IV_results)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               wage_log   R-squared:                     -0.1748
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1775
No. Observations:                2215   F-statistic:                    227.37
Date:                Tue, May 17 2022   P-value (F-stat)                0.0000
Time:                        23:46:42   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
beta_zero      2.2659     0.9999     2.2661     0.0234      0.3061      4.2257
Exper          0.1507     0.0270     5.5853     0.00

## Tentativa 2SLS FK

In [ ]:
y = dados[["wage_log"]].to_numpy()
# endog = dados[["Educ"]]
# exog = dados[['beta_zero', 'Exper', 'Exper2', 'South', 'Black']]
# instrument = dados[['near4']]

In [ ]:
# X1 nao tem a variavel endogena educ, porem considera a variavel instrumental near4
X1 = dados[['beta_zero', 'Exper', 'Exper2', 'South', 'Black', 'near4']].to_numpy()
X1

In [ ]:
# A variavel endogena vira Y
y1 = dados[["Educ"]].to_numpy()
y1

In [ ]:
#encontra o Z chapeu
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error    
import statsmodels.api as sm

# Preciso do treino e teste aqui para capturar r2
lr_modelo = LinearRegression()
lr_modelo.fit(X=X1, y=y1)
z_hat = lr_modelo.predict(X1)
z_hat

In [ ]:
# inclui as variaveis conhecidas e concatena com o Z chapeu. Ela substitui a variavel endogena. Nao inclui a variavel instrumental.
X2 = np.concatenate( (dados[['beta_zero', 'Exper', 'Exper2', 'South', 'Black']].to_numpy(), z_hat) ,axis=1)
X2 

In [ ]:
ols_model = sm.OLS(y,X2)
ols_results = ols_model.fit()
print(ols_results.summary())

## Refs

- https://www.uio.no/studier/emner/sv/oekonomi/ECON4135/h11/undervisningsmateriale/SW3e_ch12_slides%20GE.pdf
- https://epge.fgv.br/we/MFEE/Econometria/2012?action=AttachFile&do=get&target=cap10.pdf


## Aumentando o numero de instrumentos

Vamos agora instrumentalizar $Educ$ com $near4$, $near2$, $fatheduc$ e $motheduc$.

Assim, vamos estimar o modelo com uma variável endógena e quatro instrumentos, configurando o caso sobre-identificado.

Ou seja, para o indivíduo $i$, o modelo é:

\begin{equation}
y_i=x_i'\beta +u_i
\label{eq:modelxb}
\end{equation}
em que $$\underbrace{x_i}_{(K\times 1)} = (1, \underbrace{Educ_i}_{\text{endógena}}, Exper_i, Exper_i^2, South_i, Black_i)$$ 

e o **vetor de instrumentos** $$\underbrace{z_i}_{(L\times 1)} = (1, \underbrace{near4_i, near2_i,  fatheduc_i, motheduc_i}_{\text{instrumentos}}, Exper_i, Exper_i^2, South_i, Black_i),$$


<span style='color:red'>Caso que tem mais instrumentos do que variaveis endogenas. Mais complicado.</span>

com $L>K$.

Assim, o primeiro passo é construir o vetor $Z$. Neste caso é um pouco mais trabalhoso e convém montar um vetor adicional.

In [ ]:
listainstrumentos = ["near4", "near2", "fatheduc",
                     "motheduc"]  # Cria lista de variáveis instrumentais

instrumentos = np.array(dados[listainstrumentos]
                        )  # Seleciona os dados e salva numa matriz temporária

Z = np.concatenate((np.ones( (n, 1)), instrumentos, Exper2, Dados[:, 1:]),
                   axis=1)  # Concatena a constante com X_tmp1
print(len(Z), len(Z[0]) )
Z

Lembrando que no caso em que $L>K$, procedemos em dois estágios:

Procedimento de dois estágios:

1. *Primeiro Estágio*: Regredir $X$ em $Z$ e salvar os valores preditos, $\hat{X}$.
Lembre-se de que esta regressão equivale a pré-multiplicar $X$ pela matriz de projeção, neste caso baseada em $Z$:
$$ \hat{X} = Z(Z'Z)^{-1}Z'X = P_ZX$$


$P_ZX$ => Matriz de Projecao 



2. *Segundo Estágio*: Regridir $y$ em $\hat{X}$ usando o estimador de OLS:
\begin{equation}
    {\beta}_{2SLS} = (\hat{X}'\hat{X})^{-1}(\hat{X}'y)
\end{equation}

Assim, 

In [ ]:
P_z = Z@inv(Z.T@Z)@Z.T
X_hat = P_z@X

- **Avar**: A variancia Assintotica

Entretanto, a Avar fica um mais envolvida (caso de heterocesticidade):
\begin{align*}
\widehat{\text{Avar}(\beta_{2SLS})} &=     
(X'Z(Z'Z)^{-1}Z'X)^{-1}(X'Z(Z'Z)^{-1}Z \\
 &\quad \quad \quad\boldsymbol{{D}}Z(Z'Z)^{-1}Z'X)(X'Z(Z'Z)^{-1}Z'X)^{-1}\\
 &=(\hat{X}'\hat{X})^{-1} \hat{X}'\boldsymbol{{D}} \hat{X} (\hat{X}'\hat{X})^{-1}
\end{align*}

Note que, sob homocedasticidade, $\boldsymbol{D_u}=s^2I_n$ e, assim, 
\begin{align*}
\widehat{\text{Avar}(\beta_{2SLS})} &= s^2(X'Z(Z'Z)^{-1}Z'X)^{-1} \\
                                    &= s^2(\hat{X}'\hat{X})^{-1} 
\end{align*}

<span style='color:red'>No primeiro caso a ponderaçao pela correção, no segundo o caso a ponderação é sobre a covariancia do restante dos instumentos.</span>

In [ ]:
Beta_2sls = inv(X_hat.T@X_hat)@(X_hat.T@y)
e_hat     = y-X@Beta_2sls;                  # resíduo da regressão i
s2_2sls   = (e_hat.T@e_hat)/(n-k);          # estimativa de s2 para a amostra i
AVAR_2sls = s2_2sls*inv(X_hat.T@X)          # Estimando a Variância esférica
dp        = sqrt(np.diag(AVAR_2sls));       # Computa o desvio padrão
tsta_2sls = Beta_2sls.T/dp                  # Computa a estatística t.
D         = np.zeros((n,n))
for i in range(n):
    D[i,i] = e_hat[i,0]**2
    
Whit_2sls = inv(X_hat.T@X_hat)@(X_hat.T@D@X_hat)@inv(X_hat.T@X_hat)
se_r_2sls = sqrt(np.diag(Whit_2sls));
t_r_2sls  = Beta_2sls.T/se_r_2sls;

R2=1-((e_hat.T@e_hat)/n)/np.var(y)
R2
R2_adj=1-((e_hat.T@e_hat)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj

print('-----------------------------------------------------------------------------');
print('                                Estimação 2SLS');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '      Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_2sls[0],    '    Variável dependente  ', "log(Wage)")
print('-----------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat     S.E. Rob     t-Rob')
for i in range(k):
    tuplaprint=(Beta_2sls[i],dp[i],tsta_2sls[0,i],se_r_2sls[i],t_r_2sls[0,i])                # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %8.4f    %8.4f    %8.4f" % tuplaprint)  # imprime cada coeficiente

print('-----------------------------------------------------------------------------')


In [ ]:
IV_model = IV2SLS(y,X,Z)
IV_results = IV_model.fit()
print(IV_results.summary())

# 3SLS - GMM Eficiente - OLS 3 estagios
Uma vez que temos disponível uma estimativa não enviesada de $\beta_{2sls}$, podemos atingir maior eficiência se trocarmos a matriz de ponderação do GMM para a matriz 
$$W = (Z'D^{-1}Z)^{-1}$$

Com isso obtendo estimativas a partir do estimador 3SLS.

Assim, o estimador 3SLS é dado por:
    $$\hat{\beta}_{3sls}=  \bigg(X'Z(Z'\hat{D}Z)^{-1}Z'X\bigg)^{-1} X'Z(Z'\hat{D}Z)^{-1}Z'y$$

E é eficiente, pois sua matriz de variância é se simplifica para:
\begin{align*}
\widehat{\text{Var}(\hat{\beta_{3sls}}|X,Z)}  &= \Big(X'Z(Z'\hat{D}Z)^{-1}Z'X\Big)^{-1} X'Z(Z'\hat{D}Z)^{-1} \\ 
 &\times Z'\hat{D} Z(Z'\hat{D}Z)^{-1}Z'X \Big(X'Z (Z'\hat{D}Z)^{-1} Z'X\Big)^{-1}\\
 &= (X'Z(Z'\hat{D}Z)^{-1}Z'X)^{-1}
\end{align*}

Note que a matriz D associada à $\beta_{2sls}$ ainda está na memória. Assim, não precisamos recomputá-la.



In [ ]:
W = inv(Z.T @ D @ Z)

In [ ]:
Beta_3sls = inv(X.T @ Z @ W @ Z.T @ X) @ (X.T @ Z @ W @ Z.T @ y) # Computa o Beta de 3SLS
e_hat     = y-X@Beta_3sls;                  # resíduo da regressão i
s2_3sls   = (e_hat.T@e_hat)/(n-k);          # estimativa de s2 para a amostra i
AVAR_3sls = inv(X.T@Z@W@Z.T@X)              # Avar de 3SLS
dp        = sqrt(np.diag(AVAR_3sls));       # Computa o desvio padrão
tsta_3sls = Beta_3sls.T/dp                  # Computa a estatística t.
D         = np.zeros((n,n))
for i in range(n):
    D[i,i] = e_hat[i,0]**2
    
Whit_3sls = inv(X.T@Z@W@Z.T@X)              # Aqui não faz sentido, pois já foi ponderado por D.
se_r_3sls = sqrt(np.diag(Whit_3sls));
t_r_3sls  = Beta_3sls.T/se_r_3sls;

R2=1-((e_hat.T@e_hat)/n)/np.var(y)
R2
R2_adj=1-((e_hat.T@e_hat)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj

print('\n=============================================================================')
print('-----------------------------------------------------------------------------');
print('                                Estimação 3SLS');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '    Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_3sls[0],    '    Variável dependente  ', "log(Wage)")
print('\n---------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat     S.E. Rob     t-Rob')
for i in range(k):
    tuplaprint=(Beta_3sls[i],dp[i],tsta_3sls[0,i],se_r_3sls[i],t_r_3sls[0,i])                # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %8.4f    %8.4f    %8.4f" % tuplaprint)  # imprime cada coeficiente

print('\n---------------------------------------------------------------------------')


In [ ]:
IV_model   = IV2SLS(y,X,Z)
IV_results = IV_model.fit()
print(IV_results.summary())

# Teste de Endogeneidade
## Hausman-Wu
Como $L>K$, podemos testar se $Educ$ é, de fato, endógena

Para tanto, primeiro obtemos o resíduo da regressão de $Educ$ em seus instrumentos:

In [ ]:
Res_Maker  = np.eye(n)-P_z;
eta        = Res_Maker@X[:,1]
eta        = np.reshape(eta,(n,1))

In [ ]:
X_h        = np.concatenate((X,eta),axis=1)
listanomes = listanomes + ["eta"]

In [ ]:
Beta_haus = inv(X_h.T@X_h)@(X_h.T@y)
e_hat     = y-X_h@Beta_haus;               # resíduo da regressão i
s2_haus   = (e_hat.T@e_hat)/(n-k);         # estimativa de s2 para a amostra i
AVAR_haus = s2_haus*inv(X_h.T@X_h)         # Estimando a Variância esférica
dp        = sqrt(np.diag(AVAR_haus));      # Computa o desvio padrão
tsta_haus = Beta_haus.T/dp                 # Computa a estatística t.
D         = np.zeros((n,n))
for i in range(n):
    D[i,i] = e_hat[i,0]**2
    
Whit_haus = inv(X_h.T @ X_h) @ (X_h.T @ D @ X_h) @ inv(X_h.T @ X_h)
se_r_haus = sqrt(np.diag(Whit_haus));
t_r_haus  = Beta_haus.T/se_r_haus;

R2=1-((e_hat.T @ e_hat)/n)/np.var(y)
R2
R2_adj=1-((e_hat.T @ e_hat)/(n-k))/(np.var(y)*n/(n-1))               
R2_adj

print('\n=============================================================================')
print('-----------------------------------------------------------------------------');
print('                             Teste - Hauman-Wu');
print('                      S.E. homocedasticos e robustos');
print('-----------------------------------------------------------------------------');
print('R2:    ' , R2[0],    '    Graus de liberdade   ', n-k)
print('R2_adj:' , R2_adj[0],'    Número de Observações', n )
print('s^2:   ' , s2_haus[0],    '    Variável dependente  ', "log(Wage)")
print('\n---------------------------------------------------------------------------')
print('               Coef        S.E.       t-Stat     S.E. Rob     t-Rob')
for i in range(k+1):
    tuplaprint=(Beta_haus[i],dp[i],tsta_haus[0,i],se_r_haus[i],t_r_haus[0,i])                # cria uma tupla com os resultados
    print("%10s" % listanomes[i]," %8.4f    %8.4f    %8.4f    %8.4f    %8.4f" % tuplaprint)  # imprime cada coeficiente

print('\n---------------------------------------------------------------------------')


## As condições de momento do estimador de GMM são válidas?

O teste J oferece uma estatística simples para verificar a "validade" das condições de momento do GMM.



In [ ]:
e_hat = y-X @ Beta_3sls;               # resíduo da regressão do modelo \beta_3sls
W = inv((Z.T @ D @ Z));
Q_n =(np.transpose(Z.T @ e_hat)) @ W @ ((Z.T @ e_hat));
J_stat = Q_n;
print(J_stat)
from scipy.stats import chi2
L=len(Z[0])
K=len(X[0])
df = L-K
Jpvalue=1-chi2.cdf(J_stat, df)

In [ ]:
print('-----------------------------------------------------------------------------');
print('               Teste de sobre-identificação Sargan-Hansen (J)');
print('                       com K Graus de liberdade (gl)');
print('-----------------------------------------------------------------------------');
tuplaprint=(J_stat,df,Jpvalue)                       # cria uma tupla com os resultados
print()
print('                  J-Stat', '            gl', '             p-valor' );    
print("                %8.4f         %8.4f           %8.4f" % tuplaprint)
print('-----------------------------------------------------------------------------');
print('-----------------------------------------------------------------------------');
